# **Song Popularity Prediction Notebook**

This is a notebook containing a part of the analysis of my bachelor's thesis.


In [1]:
# import jupyterthemes
# !jt -l
# !jt -t onedork -T # -T to retain the toolbar

In [3]:
import pandas as pd, numpy as np
import random, os, re
import statistics
random.seed(42)

In [4]:
import warnings
warnings.filterwarnings('ignore')

In [49]:
df = pd.read_excel("out1.xlsx")
df = df.sort_values(by="uniq_id")
df = df.reset_index()
df = df.drop(columns="index")

In [ ]:
df["Rank"] = 0
i,j = 0, [x for x in range(50,0,-1)]
while i<=len(df.genre):
    df.iloc[i:i+50]["Rank"] = j
    i+=50

## Text Preprocessing

In [ ]:
for x in range(len(df.Lyrics)):
    if df.Lyrics[x]!=0:
        while df.Lyrics[x].find("[")!=-1:
            s = df.Lyrics[x]
            df.Lyrics[x] = s[:s.find("[")] +s[s.find("]")+1:]
            
df.Lyrics = df.Lyrics.map(lambda x: re.sub("r'\([^)]*\)", ' ', str(x)))
df.Lyrics = df.Lyrics.map(lambda x: re.sub('[\",-.\n!?:]', ' ', str(x)))
df.Lyrics = df.Lyrics.map(lambda x: re.sub('    ', ' ', str(x)))
df.Lyrics = df.Lyrics.map(lambda x: re.sub('   ', ' ', str(x)))
df.Lyrics = df.Lyrics.map(lambda x: re.sub('  ', ' ', str(x)))

#2-letter words + togli caratteri finali erronei dei lyrics
for x in range(len(df.Lyrics)):
    shortword = re.compile(r'\W*\b\w{1,3}\b')
    df.Lyrics[x] = shortword.sub('', df.Lyrics[x])
    if df.Lyrics[x][-32:] == ' Embed Share Url Copy Embed Copy':
        df.Lyrics[x] = df.Lyrics[x][:-32]
        
for x in range(len(df.Lyrics)):
    df.Lyrics[x] = re.sub("'s","",str(df.Lyrics[x]))
    df.Lyrics[x] = re.sub(" ' ", "", str(df.Lyrics[x]))
    df.Lyrics[x] = re.sub("'", "", str(df.Lyrics[x]))
    df.Lyrics[x] = re.sub("  ", " ", df.Lyrics[x])
    df.Lyrics[x] = re.sub("n't", "", df.Lyrics[x])
    df.Lyrics[x] = re.sub('["“”’–]', "", df.Lyrics[x])
    df.Lyrics[x] = re.sub('[()]', "", df.Lyrics[x])
    if "Rap Genius Trends" in df.loc[i, "Lyrics"]:
        df.loc[i, "Lyrics"] = 0

In [ ]:
from gensim.utils import simple_preprocess
from nltk.corpus import wordnet, stopwords
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer
from os.path import expanduser

In [ ]:
# Part-of-Speech Tagging
def wordnet_pos_gen(lista):
    pos_tags = list(nltk.pos_tag(lista))
    tag_dict = {"J": wordnet.ADJ, "N": wordnet.NOUN, "V": wordnet.VERB, "R": wordnet.ADV}
    t = []
    for x in pos_tags:
        try:
            t.append((x[0], tag_dict[x[1][0]]))
        except Exception:
            t.append(
                (x[0], "n"))  # a get around to assign irrelevant tags to one of wordnet classes, noun in this case
    return t

# Stopping
def stopping(ls, *args):
    processed = []
    for case in ls:
        tok = word_tokenize(case)
        stop_words = set(stopwords.words("english"))
        stop_words.add(","), stop_words.add("."), stop_words.add(";"), stop_words.add(":")
        res = [x for x in tok if not x in stop_words]
        result = ""
        for x in res:
            result = result + " " + x
        processed.append(result)
    return processed

# Lemmatizer
def lemma(lista):
    out = []
    t = wordnet_pos_gen(lista)
    lemmatizer = WordNetLemmatizer()
    res = [lemmatizer.lemmatize(w[0], w[1]) for w in t]
    result = ""
    for x in res:
        result = result + " " + x
    out.append(result.lower())  # case normalization step here!
    return out

In [ ]:
lyric_corpus_tokenized = []
tokenizer = RegexpTokenizer(r'\w+')
for lyric in df.Lyrics:
    tokenized_lyric = tokenizer.tokenize(lyric.lower())
    lyric_corpus_tokenized.append(tokenized_lyric)

for s,song in enumerate(lyric_corpus_tokenized):
    filtered_song = []
    for token in song:
        if len(token) > 2 and not token.isnumeric():
            filtered_song.append(token)
    lyric_corpus_tokenized[s] = filtered_song

for y in range(len(df.Lyrics)):
    print(y)
    df.Lyrics[y] = [x for x in stopping(lemma(lyric_corpus_tokenized[y])) if x != ""]

for x in range(len(df.Lyrics)):
    s = ""
    for y in df.Lyrics[x]:
        s+=y
    df.Lyrics[x] = s

df.to_excel("out2.xlsx")

## Feature Engineering (for Text)

In [52]:
# Top 100 words
wordCorpus = {}
for i in range(len(df)):
    s = str(df.loc[i, "Lyrics"]).split()
    for w in s:
        if w in ["i", "you", "she", "he", "we", "they"]:
            continue
        elif w in wordCorpus.keys():
            wordCorpus[w] += 1
        else:
            wordCorpus[w] = 1

top100Words = sorted(wordCorpus, key = wordCorpus.get, reverse = True)[:100]

#Add the top 100 words as columns to df and measure TF for each row
for col in top100Words:
    df[col] = 0

for i in range(len(df)):
    TF_perRow = {}
    s = str(df.loc[i, "Lyrics"]).split()
    for w in s:
        if w not in top100Words:
            continue
        elif w in TF_perRow.keys():
            TF_perRow[w] += 1
        else:
            TF_perRow[w] = 1
    for w in TF_perRow.keys():
        df.loc[i, w] = TF_perRow[w]

In [ ]:
# The number of times a song appeared on the charts
d = df.groupby("song")["song"].count().to_dict()
df["times_charted"] = [d[x] for x in df.song]

In [ ]:
# The number of different genres a song belongs to
d = df.groupby(["song", "genre"])["genre"].unique().groupby("song").count().to_dict()
df["num_genres"] = [d[x] for x in df.song]

In [ ]:
#Output dataframe in order to perform LDA in R!
df.to_excel("PyToR.xlsx")

In [ ]:
#LDA w Gibbs Sampling at 5000 iterations performed in R!
df1 = pd.read_excel("RToPy.xlsx", engine='openpyxl') 
df[["lda10topic{}".format(x) for x in range(1,11)]] = df1[["{}".format(str(x)) for x in range(1,11)]]

In [ ]:
songTopicComp = []
for x in range(len(df.uniq_id)):
    songTopicComp.append(list(df[["lda10topic{}".format(x) for x in range(1,11)]].iloc[x]))

genres = ["Dan", "Pop", "Rap", "RnB", "Roc", "Chr", "Cou"]
dic_genres = {"Dan":0, "Pop":1, "Rap":2, "RnB":3, "Roc":4, "Chr":5, "Cou":6}
avgTopComp = []

for x in genres:
    aa = [songTopicComp[y] for y in range(len(df.genre)) if df.genre[y]==x]
    temp=[]
    for y in range(10):
        temp.append(statistics.mean([aa[z][y] for z in range(len(aa))]))
    avgTopComp.append(temp)

lsm = []
for x in range(len(df.genre)):
    lsm.append(
        statistics.mean(list(np.divide(abs(np.array(songTopicComp[x]) - np.array(avgTopComp[dic_genres[df.genre[x]]])),
                           np.array(songTopicComp[x]) + np.array(avgTopComp[dic_genres[df.genre[x]]]) + 0.0001))))
df["lsm"] = lsm

In [ ]:
df.to_excel("out3.xlsx")

## Preprocessing (for Audio)¶

In [ ]:
import librosa, scipy, pickle

In order to make Librosa work *ffmpeg* must be installed (otherwise *librosa.load()* would raise a *audioread.exceptions.NoBackendError*). 

In Ubuntu it is enough to run "*sudo apt install ffmpeg*" in terminal.


In [ ]:
os.chdir("Tracks/")

#Cast Song Title from float (to int) to string
floatRowsMask = np.where([isinstance(x, int) for x in  df.song])
df.song.iloc[floatRowsMask] = df.loc[floatRowsMask, "song"].astype('str')

#Rename all files in Tracks folder to lower case
path = "/home/alessio/Desktop/University/Git/thesis/proj/Tracks/"
for file in os.listdir(path):
    os.rename(path + file, path + file.lower())

for x in range(len(df.date)):  
    song_temp = df["song"][x]
    print(song_temp)
    song_temp = re.sub("[/:\"]", "", song_temp)
    song_temp = re.sub("[\?\*]", "#", song_temp)
    song_temp = re.sub("腎", "t", song_temp)
    song_temp = song_temp.lower()
    if song_temp.find("(") != -1: song_temp = song_temp[:song_temp.find("(") - 1] + song_temp[song_temp.find(")") + 1:]
    df["song"][x] = re.sub("ac/dc", "acdc", song_temp)
    artist_temp = df["artist"][x]
    artist_temp = re.sub("[/:\"]", "", artist_temp)
    artist_temp = re.sub("[\?\*]", "#", artist_temp)
    artist_temp = artist_temp.lower()
    df["artist"][x] = re.sub("ac/dc", "acdc", artist_temp)

#Control loop to check if error occurs (ie. song not found in folder)
for x in range(len(df.date)):
    b = df["song"][x] 
    if b.find("(") != -1:
      b = b[:b.find("(")-1] + b[b.find(")")+1:]  
      df["song"][x] = b
    a = b + " - " + df["artist"][x]
    if a.lower() in [x[:-4] for x in os.listdir()]:
        pass
    else:
        print(a)
        print(df["artist_song"][x])

## Feature Engineering (for Audio)

### Chroma

In [ ]:
j_chroma = range(3, 9)
hop_length = 512
def kl_divergence(p, q):
    return np.sum(np.where(p != 0, p * np.log(p / q), 0))
def d(s1, s2):
    M = (s1 + s2) / 2
    return (kl_divergence(s1, M) + kl_divergence(s2, M)) / 2
def s_eff(a,b):
    return matrixC[b] - matrixC[a]

def eucl_dist(a,b):
    return np.linalg.norm(a-b)

In [ ]:
scChromaOUT = []
for ii in range(len(df.date)):
    if df.song[ii] not in [x[0] for x in scChromaOUT]:
        print(df.song[ii],ii)
        x, sr = librosa.load("{} - {}.mp3".format(df.song[ii],df.artist[ii]))
        chromagram = pd.DataFrame(librosa.feature.chroma_stft(x, sr=sr, hop_length=hop_length, )).transpose()
        matrixC = []
        for i in range(len(chromagram[1])): #quell'[1] che sarebbe la prima corda conta solo per il range!
            matrixC.append(sum(np.array(chromagram.iloc[0:i+1])))
        scChroma = []
        v_eff=[]
        for j in j_chroma:
            w = 2 ** (j - 1)
            v_eff = []
            print(j)
            for i in range(len(chromagram[1])):
                #print(i)
                if i>w and i<len(chromagram[1])-w: #l'[1] conta sempre e solo per il range, le corde le faccio tutte
                    v_eff.append(d(s_eff(i - w + 1, i - 1), s_eff(i, i + w)))
                else:
                    v_eff.append(0)
            scChroma.append(statistics.mean(v_eff))
        scChromaOUT.append((df.song[ii],df.artist[ii],scChroma))


In [ ]:
import pickle
with open('done2.pkl', 'wb') as f:
  pickle.dump(scChromaOUT, f)

df["Chroma1"],df["Chroma2"],df["Chroma3"],df["Chroma4"],df["Chroma5"],df["Chroma6"]=0.0,0.0,0.0,0.0,0.0,0.0
for x in range(len(df.date)):
    try:
        el=[y for y in scChromaOUT if y[0]==df.song[x]][0][-1]
        df["Chroma1"][x],df["Chroma2"][x],df["Chroma3"][x],df["Chroma4"][x],df["Chroma5"][x],df["Chroma6"][x]=el
    except:
        print(df.song[x])

### Timbre 

In [ ]:
import time
sr=44100
win_len = round(sr * 2.97)
win = np.hamming(win_len)
j_timbre = range(1, 7)

In [ ]:
scTimbreOUT = []
for ii in range(len(df.date)):
    if df.song[ii] not in [x[0] for x in scTimbreOUT]:
        start_time = time.time()
        print(df.song[ii],ii)
        x, sr = librosa.load("{} - {}.mp3".format(df.song[ii],df.artist[ii]),sr)
        d = round(librosa.get_duration(x, sr=sr)) 
        i, ls = 0, []
        while i < d * sr:
            a = x[i:i + win_len]  # win_len = datapoints in that window interval, +i <-- il movimento di 1 sec, (+0,+44100, +44100*2, ecc)
            if len(x[i:i + win_len]) < win_len: #per l'ultimo pezzo magari non lungo abbastanza
                a = np.array(list(x[i:i + win_len]) + [0 for x in range(win_len - len(x[i:i + win_len]))])
            windowed_segment = win * a
            ls_t = []
            for y in range(0, len(windowed_segment), 512):
                ls_t.append([x[0] for x in librosa.feature.mfcc(windowed_segment[y:y + 512], sr=sr, n_mfcc=36).tolist()]) #var[0:x] con x>len(var)] funge lo stesso :/
            df1 = pd.DataFrame(ls_t).transpose()
            ls.append([statistics.mean(df1.iloc[x]) for x in range(len(df1[35]))])
            i += sr
        df1 = pd.DataFrame(ls)

        matrixC = []
        for i in range(len(df1[35])):
            matrixC.append(sum(np.array(df1.iloc[0:i+1])))
        scTimbre = []
        v_eff=[]
        for j in j_timbre:
            w = 2 ** (j - 1)
            v_eff = []
            print(j)
            for i in range(len(df1[35])):
                if i>w and i<len(df1[35])-w:
                    v_eff.append(eucl_dist(s_eff(i - w + 1, i - 1), s_eff(i, i + w))) #kl div serve per le prob distro, non è quello che cerco, uso la eucledian distance https://groups.google.com/g/librosa/c/9Xs15kDImOg
                else:
                    v_eff.append(0)
            scTimbre.append(statistics.mean(v_eff))
        scTimbreOUT.append((df.song[ii],df.artist[ii],scTimbre))
        print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
df["Timbre1"],df["Timbre2"],df["Timbre3"],df["Timbre4"],df["Timbre5"],df["Timbre6"]=0.0,0.0,0.0,0.0,0.0,0.0
for x in range(len(df.date)):
    try:
        el=[y for y in scTimbreOUT if y[0]==df.song[x] and y[1]==df.artist[x]][0][-1]
        df["Timbre1"][x],df["Timbre2"][x],df["Timbre3"][x],df["Timbre4"][x],df["Timbre5"][x],df["Timbre6"][x]=el
    except:
        print(df.song[x])

### Arousal 

In [ ]:
sr=44100
win_len = round(sr * 2.97)
win = np.hamming(win_len)

In [ ]:
arousalOUT = []
for ii in range(len(df.date)):
    if (df.song[ii], df.artist[ii]) not in [(x[0],x[1]) for x in arousalOUT]:
        print(df.song[ii], ii)
        x, sr = librosa.load("{} - {}.mp3".format(df.song[ii], df.artist[ii]), sr)
        d = round(librosa.get_duration(x, sr=44100))  
        i, arousal = 0, []
        while i < d * sr:
            a = x[i:i + win_len]  # win_len = datapoints in that window interval, +i <-- il movimento di 1 sec, (+0,+44100, +44100*2, ecc)
            if len(x[i:i + win_len]) < win_len:
                a = np.array(list(x[i:i + win_len]) + [0 for x in range(win_len - len(x[i:i + win_len]))])
            windowed_segment = win * a
            arousal.append(np.sum(np.abs(windowed_segment)))
            i += sr
        arousalOUT.append((df.song[ii],df.artist[ii],statistics.mean(arousal),statistics.stdev(arousal)))

In [ ]:
df["ArousalMean"],df["ArousalSD"]=0.0,0.0
for x in range(len(df.date)):
    try:
        el = [y for y in arousalOUT if y[0]==df.song[x]][0]
        df["ArousalMean"][x],df["ArousalSD"][x] = el[2], el[3]
    except:
        print(df.song[x])

### MFCC 

In [ ]:
sr = 44100
win_len = round(sr * 0.025)

In [ ]:
mfcc = []
for ii in range(len(df.date)):
    print(ii)
    if (df.song[ii], df.artist[ii]) not in [(x[0], x[1]) for x in mfcc]:
        print(df.song[ii],ii)
        x, sr = librosa.load("{} - {}.mp3".format(df.song[ii],df.artist[ii]),sr)
        i, ls = 0, []
        d = round(librosa.get_duration(x, sr=sr))
        while i < d * sr:
           windowed_segment = x[i:i + win_len]  # win_len = datapoints in that window interval, +i <-- il movimento di 1 sec, (+0,+44100, +44100*2, ecc)
           if len(x[i:i + win_len]) < win_len:
               windowed_segment = np.array(
                   list(x[i:i + win_len]) + [0.0 for x in range(win_len - len(x[i:i + win_len]))])
           ls.append(librosa.feature.mfcc(windowed_segment, sr=sr, n_mfcc=20))
           i += round(0.015 * sr)

        llss = []
        for x in range(len(ls)):
           for y in range(20):
               llss.append(ls[x][y][0]), llss.append(ls[x][y][1]), llss.append(ls[x][y][2])

        i, final = 0, []
        while i < len(llss):
           final.append([llss[x + i] for x in range(1, 60, 3)])
           final.append([llss[x + i] for x in range(2, 60, 3)])
           final.append([llss[x + i] for x in range(3, 60, 3)])
           i += 60

        df1 = pd.DataFrame(final)
        mfcc.append((df.song[ii], df.artist[ii],df.genre[ii],[np.mean(df1[x]) for x in df1.columns]))  # credo che io debba fare l'average, il df di una singola canzone a 55k rows, se ce ne metto altre 4199 --> 230945000

In [ ]:
df["toDropMFCC"]="Zero"
for x in range(len(df.date)):
    try:
        el = [y for y in mfcc if y[0]==df.song[x] and y[1]==df.artist[x]][0][-1] 
        df["toDropMFCC"][x] = el
    except:
        print(df.song[x])
        
from sklearn.cluster import KMeans
mfccOUT=[]
for g in list(set([x[2] for x in mfcc])):
    kmeans = KMeans(n_clusters=32, random_state=42).fit(list(df.iloc[[x for x in range(len(df.date)) if df.genre[x]==g]].toDropMFCC))
    mfccOUT.append((g,kmeans.cluster_centers_))

for x in range(20): 
    df["mfcc{}".format(x)]=0.0

from scipy.spatial import distance
def findTheRightCentroid(mfcc, centroid): 
    l=[(distance.euclidean(mfcc,centroid[x]),x) for x in range(32)]
    spotMin = min([x[0] for x in l])
    return [centroid[x[1]] for x in l if x[0]==spotMin][0]

for x in range(len(df.date)):
    rightCentroid = findTheRightCentroid(df["toDropMFCC"][x], [y[1].tolist() for y in mfccOUT if df.genre[x]==y[0]][0]) #i pass the row + the right centroids wrt song genre
    df["mfcc0"][x],df["mfcc1"][x],df["mfcc2"][x],df["mfcc3"][x],df["mfcc4"][x],df["mfcc5"][x],df["mfcc6"][x],df["mfcc7"][x],df["mfcc8"][x],df["mfcc9"][x],df["mfcc10"][x],df["mfcc11"][x],df["mfcc12"][x],df["mfcc13"][x],df["mfcc14"][x],df["mfcc15"][x],df["mfcc16"][x],df["mfcc17"][x],df["mfcc18"][x],df["mfcc19"][x] = rightCentroid

In [ ]:
df.to_excel("out4.xlsx")

## Berger et al. 2020 Models

In [47]:
df = pd.read_excel("out4.xlsx")
df.head(5)

,Unnamed: 0,uniq_id,date,month,year,d2014,d2015,d2016,q1_14,q2_14,...,mfcc10,mfcc11,mfcc12,mfcc13,mfcc14,mfcc15,mfcc16,mfcc17,mfcc18,mfcc19
0,0,1,2014-02-08,2,2014,1,0,0,1,0,...,0.431868,-2.027841,-0.561371,-2.312961,2.193644,-1.622512,0.348201,-1.994333,1.703286,-0.347439
1,1,2,2014-02-08,2,2014,1,0,0,1,0,...,4.199413,-1.006321,2.914582,-0.881443,1.930117,-0.162764,0.057076,-0.522933,1.375426,0.228533
2,2,3,2014-02-08,2,2014,1,0,0,1,0,...,6.790519,1.208557,5.410130,0.837216,2.651076,2.752934,2.209173,2.334909,1.669348,1.966537
3,3,4,2014-02-08,2,2014,1,0,0,1,0,...,5.119591,0.188172,4.029795,-0.362633,4.020254,1.348178,1.388804,-0.276839,1.731131,-1.774836
4,4,5,2014-02-08,2,2014,1,0,0,1,0,...,6.790519,1.208557,5.410130,0.837216,2.651076,2.752934,2.209173,2.334909,1.669348,1.966537


### Model 1

In [49]:
import statsmodels.formula.api as smf
results = smf.ols('Rank ~ you', data = df).fit().summary()
print(results)

                            OLS Regression Results                            
Dep. Variable:                   Rank   R-squared:                       0.002
Model:                            OLS   Adj. R-squared:                  0.002
Method:                 Least Squares   F-statistic:                     9.433
Date:                Wed, 22 Feb 2023   Prob (F-statistic):            0.00215
Time:                        01:39:00   Log-Likelihood:                -17166.
No. Observations:                4200   AIC:                         3.434e+04
Df Residuals:                    4198   BIC:                         3.435e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     24.6839      0.347     71.227      0.0

### Model 2

In [50]:
#Build "times charted" column
df["times_charted"] = 0
d = df.groupby(['artist_song'])['artist_song'].count().to_dict() 
for i in range(len(df.artist_song)-1):
  df.loc[i, "times_charted"] = d[df.artist_song[i]]

results = smf.ols('Rank ~ you + times_charted + count_genres + d_airplaysong', data=df).fit().summary()
print(results)

                            OLS Regression Results                            
Dep. Variable:                   Rank   R-squared:                       0.107
Model:                            OLS   Adj. R-squared:                  0.106
Method:                 Least Squares   F-statistic:                     126.0
Date:                Wed, 22 Feb 2023   Prob (F-statistic):          1.10e-101
Time:                        01:39:05   Log-Likelihood:                -16933.
No. Observations:                4200   AIC:                         3.388e+04
Df Residuals:                    4195   BIC:                         3.391e+04
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
Intercept        15.3216      0.713     21.474

### Model 3

In [51]:
results = smf.mixedlm("Rank ~ you", df, groups=df["artist_song"]).fit().summary()
print(results)

          Mixed Linear Model Regression Results
Model:            MixedLM Dependent Variable: Rank       
No. Observations: 4200    Method:             REML       
No. Groups:       1885    Scale:              168.4253   
Min. group size:  1       Likelihood:         -17090.5295
Max. group size:  20      Converged:          Yes        
Mean group size:  2.2                                    
----------------------------------------------------------
           Coef.   Std.Err.    z     P>|z|  [0.025  0.975]
----------------------------------------------------------
Intercept  24.200     0.412  58.729  0.000  23.393  25.008
you         0.001     0.001   1.993  0.046   0.000   0.003
Group Var  38.655     0.357                               



### Model 4

In [52]:
import statsmodels.api as sm
my_cols = ["you","g_dan","g_cou","g_chr","g_rap","g_pop","g_roc","g_rnb"]
X = sm.add_constant(df[my_cols])
results = sm.OLS(df.Rank, X).fit().summary()
print(results)

                            OLS Regression Results                            
Dep. Variable:                   Rank   R-squared:                       0.002
Model:                            OLS   Adj. R-squared:                  0.001
Method:                 Least Squares   F-statistic:                     1.386
Date:                Wed, 22 Feb 2023   Prob (F-statistic):              0.206
Time:                        01:39:12   Log-Likelihood:                -17166.
No. Observations:                4200   AIC:                         3.435e+04
Df Residuals:                    4192   BIC:                         3.440e+04
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         21.5769      0.306     70.487      0.0

### Model 5

In [53]:
my_cols = ["you",'q1_14','q2_14','q3_14','q4_14','q1_15','q2_15','q3_15','q4_15','q1_16','q2_16','q3_16','q4_16']
X = sm.add_constant(df[my_cols])
results = sm.OLS(df.Rank, X).fit().summary()
print(results)

                            OLS Regression Results                            
Dep. Variable:                   Rank   R-squared:                       0.002
Model:                            OLS   Adj. R-squared:                 -0.001
Method:                 Least Squares   F-statistic:                    0.7853
Date:                Wed, 22 Feb 2023   Prob (F-statistic):              0.666
Time:                        01:39:12   Log-Likelihood:                -17166.
No. Observations:                4200   AIC:                         3.436e+04
Df Residuals:                    4187   BIC:                         3.444e+04
Df Model:                          12                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         22.7839      0.320     71.094      0.0

### Model 6

In [54]:
my_cols = ["you"] + ["lda10topic{}".format(x) for x in range(1,11)] 
X = sm.add_constant(df[my_cols])
results = sm.OLS(df.Rank, X).fit().summary()
print(results)

                            OLS Regression Results                            
Dep. Variable:                   Rank   R-squared:                       0.008
Model:                            OLS   Adj. R-squared:                  0.005
Method:                 Least Squares   F-statistic:                     2.923
Date:                Wed, 22 Feb 2023   Prob (F-statistic):           0.000745
Time:                        01:39:16   Log-Likelihood:                -17155.
No. Observations:                4200   AIC:                         3.433e+04
Df Residuals:                    4188   BIC:                         3.441e+04
Df Model:                          11                                         
Covariance Type:            nonrobust                                         
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
const           24.8421      0.986     25.194   

### Model 7

In [55]:
my_cols = ["you","cogproc","affect1","percept","drives","time","relativ","informal"] 
X = sm.add_constant(df[my_cols])
results = sm.OLS(df.Rank, X).fit().summary()
print(results)

                            OLS Regression Results                            
Dep. Variable:                   Rank   R-squared:                       0.005
Model:                            OLS   Adj. R-squared:                  0.003
Method:                 Least Squares   F-statistic:                     2.572
Date:                Wed, 22 Feb 2023   Prob (F-statistic):            0.00847
Time:                        01:39:19   Log-Likelihood:                -17161.
No. Observations:                4200   AIC:                         3.434e+04
Df Residuals:                    4191   BIC:                         3.440e+04
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         24.8778      0.881     28.252      0.0

### Model 8

In [56]:
my_cols = ["you"] +  list(df.columns)[65:-37]
df=df.fillna(0) 
X = sm.add_constant(df[my_cols])
results = sm.OLS(df.Rank, X).fit().summary()
print(results)

                            OLS Regression Results                            
Dep. Variable:                   Rank   R-squared:                       0.048
Model:                            OLS   Adj. R-squared:                  0.025
Method:                 Least Squares   F-statistic:                     2.089
Date:                Wed, 22 Feb 2023   Prob (F-statistic):           3.25e-09
Time:                        01:39:21   Log-Likelihood:                -17069.
No. Observations:                4200   AIC:                         3.434e+04
Df Residuals:                    4101   BIC:                         3.496e+04
Df Model:                          98                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         28.7632      0.755     38.115      0.0

### Model 9

In [69]:
df["Intercept"]=1
my_cols =  ["times_charted","count_genres","d_airplaysong"]+list(df.columns)[65:-38]+["cogproc","affect1","percept","drives","time","relativ","informal"]+["lda10topic{}".format(x) for x in range(1,11)]+['q1_14', 'q2_14','q3_14','q4_14','q1_15','q2_15','q3_15','q4_15','q1_16','q2_16','q3_16','q4_16']+["g_dan","g_cou","g_chr","g_rap","g_pop","g_roc","g_rnb"]
md = sm.MixedLM(df.Rank, df[["Intercept", "you",] +my_cols], groups=df["artist_song"])#, exog_re=df["Intercept"])
mdf = md.fit()
print(mdf.summary())

/home/alessio/anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:512: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
/home/alessio/anaconda3/lib/python3.7/site-packages/statsmodels/regression/mixed_linear_model.py:2059: ConvergenceWarning: Retrying MixedLM optimization with lbfgs
  ConvergenceWarning)
/home/alessio/anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:512: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
/home/alessio/anaconda3/lib/python3.7/site-packages/statsmodels/regression/mixed_linear_model.py:2059: ConvergenceWarning: Retrying MixedLM optimization with cg
  ConvergenceWarning)
/home/alessio/anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:512: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", Con

                   Mixed Linear Model Regression Results
Model:                  MixedLM       Dependent Variable:       Rank       
No. Observations:       4200          Method:                   REML       
No. Groups:             1885          Scale:                    140.9180   
Min. group size:        1             Likelihood:               -16723.6713
Max. group size:        20            Converged:                No         
Mean group size:        2.2                                                
---------------------------------------------------------------------------
               Coef.     Std.Err.     z    P>|z|     [0.025       0.975]   
---------------------------------------------------------------------------
Intercept     -250.328 20033084.702 -0.000 1.000 -39264374.843 39263874.186
you              0.001        0.001  1.673 0.094        -0.000        0.003
times_charted    0.856        0.116  7.410 0.000         0.630        1.083
count_genres     6.159        0

## Berger et al. 2018 Models

In [70]:
songTopicComp = []
for x in range(len(df.uniq_id)):
    songTopicComp.append(list(df[["lda10topic{}".format(x) for x in range(1,11)]].iloc[x]))

genres = ["Dan", "Pop", "Rap", "RnB", "Roc", "Chr", "Cou"]
dic_genres = {"Dan":0, "Pop":1, "Rap":2, "RnB":3, "Roc":4, "Chr":5, "Cou":6}
avgTopComp = []

for x in genres:
    aa = [songTopicComp[y] for y in range(len(df.genre)) if df.genre[y]==x]
    temp=[]
    for y in range(10):
        temp.append(statistics.mean([aa[z][y] for z in range(len(aa))]))
    avgTopComp.append(temp)

lsm = []
for x in range(len(df.genre)):
    lsm.append(
        statistics.mean(list(np.divide(abs(np.array(songTopicComp[x]) - np.array(avgTopComp[dic_genres[df.genre[x]]])),
                           np.array(songTopicComp[x]) + np.array(avgTopComp[dic_genres[df.genre[x]]]) + 0.0001)))
    )
df["lsm"] = lsm

### Model 1

In [71]:
import statsmodels.formula.api as smf
results = smf.ols('Rank ~ lsm', data=df).fit().summary()
print(results)

                            OLS Regression Results                            
Dep. Variable:                   Rank   R-squared:                       0.001
Model:                            OLS   Adj. R-squared:                  0.001
Method:                 Least Squares   F-statistic:                     6.238
Date:                Wed, 22 Feb 2023   Prob (F-statistic):             0.0125
Time:                        01:49:24   Log-Likelihood:                -17168.
No. Observations:                4200   AIC:                         3.434e+04
Df Residuals:                    4198   BIC:                         3.435e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     22.0237      1.410     15.625      0.0

### Model 2

In [73]:
import re
a = list(df.artist)
processed=[]
exclusions = '|'.join([" with ", " featuring ", " feat ", " & ", ", ", " ft. ", " ft ", " and ", " \+ ", " x "])
for x in a:
    x = re.sub(exclusions, "|", x.lower())
    x = re.sub("  ", "", x)
    processed.append(x.split("|"))
totalCols = []
for x in processed:
    for y in x:
        totalCols.append(y)
totalCols = set(totalCols)
oldCols = len(list(df.columns))
for x in totalCols:
    df[x] = 0
for x in range(len(df.artist)):
    for y in processed[x]:
        df[y][x] = 1
artists = list(df.columns)[oldCols:]

time = ['q1_14', 'q2_14','q3_14','q4_14','q1_15','q2_15','q3_15','q4_15','q1_16','q2_16','q3_16','q4_16']

date_dic = {'q1_14':1, 'q2_14':2,'q3_14':3, 'q4_14':4, 'q1_15':5, 'q2_15':6, 'q3_15':7, 'q4_15':8, 'q1_16':9, 'q2_16':10, 'q3_16':11, 'q4_16':12}
df["date"] = 0
for x in list(date_dic.keys()):
    for y in range(len(df.genre)):
        if df[x][y] == 1:
            df["date"][y] = date_dic[x]

my_cols = ["lsm", "d_airplaysong", "count_genres", "times_charted"] + time + artists + ["lda10topic{}".format(x) for x in range(1,11)]


In [74]:
import statsmodels.api as sm
X = sm.add_constant(df[my_cols])
results = sm.OLS(df.Rank, X).fit()
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                   Rank   R-squared:                       0.120
Model:                            OLS   Adj. R-squared:                  0.114
Method:                 Least Squares   F-statistic:                     22.67
Date:                Wed, 22 Feb 2023   Prob (F-statistic):           1.15e-96
Time:                        01:49:50   Log-Likelihood:                -16904.
No. Observations:                4200   AIC:                         3.386e+04
Df Residuals:                    4174   BIC:                         3.402e+04
Df Model:                          25                                         
Covariance Type:            nonrobust                                         
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
const             9.9951      1.747      5.722

### Model 3

In [86]:
artists = [x for x in df.columns][201:]

In [87]:
my_cols = ["lsm", "d_airplaysong", "count_genres", "times_charted",] + time  + ["lda10topic{}".format(x) for x in range(1,11)] + ["WC", "cogproc", "percept", "drives", "relativ", "time", "affect1"] +artists

import statsmodels.api as sm
X = sm.add_constant(df[my_cols])
results = sm.OLS(df.Rank, X).fit()
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                   Rank   R-squared:                       0.417
Model:                            OLS   Adj. R-squared:                  0.238
Method:                 Least Squares   F-statistic:                     2.326
Date:                Wed, 22 Feb 2023   Prob (F-statistic):           6.88e-69
Time:                        01:56:21   Log-Likelihood:                -16037.
No. Observations:                4200   AIC:                         3.405e+04
Df Residuals:                    3211   BIC:                         4.033e+04
Df Model:                         988                                         
Covariance Type:            nonrobust                                         
                                       coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------------
const   

## Lee et al. Models

In [88]:
# Chroma alone
import statsmodels.api as sm
my_cols = ["Chroma1","Chroma2","Chroma3","Chroma4","Chroma5","Chroma6",]
X = sm.add_constant(df[my_cols])
results = sm.OLS(df.Rank, X).fit().summary()
print(results)


                            OLS Regression Results                            
Dep. Variable:                   Rank   R-squared:                       0.004
Model:                            OLS   Adj. R-squared:                  0.003
Method:                 Least Squares   F-statistic:                     2.807
Date:                Wed, 22 Feb 2023   Prob (F-statistic):            0.00998
Time:                        01:58:24   Log-Likelihood:                -17162.
No. Observations:                4200   AIC:                         3.434e+04
Df Residuals:                    4193   BIC:                         3.438e+04
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         21.2547      3.911      5.434      0.0

In [89]:
# Timbre alone
import statsmodels.api as sm
my_cols = ["Timbre1","Timbre2","Timbre3","Timbre4","Timbre5","Timbre6",]
X = sm.add_constant(df[my_cols])
results = sm.OLS(df.Rank, X).fit().summary()
print(results)

                            OLS Regression Results                            
Dep. Variable:                   Rank   R-squared:                       0.006
Model:                            OLS   Adj. R-squared:                  0.005
Method:                 Least Squares   F-statistic:                     4.386
Date:                Wed, 22 Feb 2023   Prob (F-statistic):           0.000200
Time:                        01:58:33   Log-Likelihood:                -17158.
No. Observations:                4200   AIC:                         3.433e+04
Df Residuals:                    4193   BIC:                         3.437e+04
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         23.5964      1.489     15.849      0.0

In [90]:
# Arousal alone
import statsmodels.api as sm
my_cols = ["ArousalSD","ArousalMean"]
X = sm.add_constant(df[my_cols])
results = sm.OLS(df.Rank, X).fit().summary()
print(results)

                            OLS Regression Results                            
Dep. Variable:                   Rank   R-squared:                       0.007
Model:                            OLS   Adj. R-squared:                  0.006
Method:                 Least Squares   F-statistic:                     14.54
Date:                Wed, 22 Feb 2023   Prob (F-statistic):           5.11e-07
Time:                        01:58:40   Log-Likelihood:                -17156.
No. Observations:                4200   AIC:                         3.432e+04
Df Residuals:                    4197   BIC:                         3.434e+04
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
const          23.2048      0.652     35.596      

In [91]:
#Mfcc alone
import statsmodels.api as sm
my_cols = ["mfcc{}".format(x) for x in range(20)] + ["Chroma1","Chroma2","Chroma3","Chroma4","Chroma5","Chroma6",] +["ArousalSD","ArousalMean"] + ["Timbre1","Timbre2","Timbre3","Timbre4","Timbre5","Timbre6",]
X = sm.add_constant(df[my_cols])
results = sm.OLS(df.Rank, X).fit().summary()
print(results)

                            OLS Regression Results                            
Dep. Variable:                   Rank   R-squared:                       0.021
Model:                            OLS   Adj. R-squared:                  0.013
Method:                 Least Squares   F-statistic:                     2.582
Date:                Wed, 22 Feb 2023   Prob (F-statistic):           1.46e-06
Time:                        01:58:46   Log-Likelihood:                -17127.
No. Observations:                4200   AIC:                         3.432e+04
Df Residuals:                    4165   BIC:                         3.455e+04
Df Model:                          34                                         
Covariance Type:            nonrobust                                         
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
const          35.5735     10.478      3.395      

# RQ 2

In [ ]:
# Random Forest Model
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import cross_val_score
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn import preprocessing

my_cols= my_cols + ["lsm"]+["Chroma1","Chroma2","Chroma3","Chroma4","Chroma5","Chroma6",] +["ArousalSD","ArousalMean"] + ["Timbre1","Timbre2","Timbre3","Timbre4","Timbre5","Timbre6",] + list(df.columns)[-1126-1885:-1126]   
my_cols.append("Rank")
df1 = df[my_cols]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df1.drop(columns=["Rank"]), df1["Rank"], test_size=0.33, random_state=42)

# Grid for GridSearchCV
grid_rf = {'n_estimators': [200, 1000],
           'max_depth': [10, 100],
           'min_samples_split': [2, 10],
           'min_samples_leaf': [1, 4],
           'bootstrap': [True, False]}

# GridSearch and Model Fit
something_rf = GridSearchCV(RandomForestRegressor(), param_grid=grid_rf, cv=5, verbose=2)
something_rf.fit(X_train, y_train)
something_rf.best_params_ #{'bootstrap': True, 'max_depth': 10, 'min_samples_leaf': 1, 'min_samples_split': 10, 'n_estimators': 1000}
predictions = something.predict(X_test)
errors = abs(predictions - y_test)

mean_squared_error(y_test, predictions, squared=True)
mean_absolute_error(y_test, predictions)
scores_rf = cross_val_score(
    RandomForestRegressor(random_state=42, n_estimators=1000, min_samples_split=2, min_samples_leaf=1, max_depth=100,
                          bootstrap=True), df.drop(columns=["cost"]), df["cost"],cv=10)  # df.drop(columns=["cost"]), df["cost"], cv=10)
scores_rf.mean()

In [ ]:
#SVM
from sklearn.preprocessing import StandardScaler
X, y = df1.drop(columns=["Rank"]), df1["Rank"]
sc_X = StandardScaler()
sc_y = StandardScaler()
X = sc_X.fit_transform(X)
y = sc_y.fit_transform(np.array(y).reshape(-1,1))

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state=42)

from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
regressor = SVR(kernel='rbf')
regressor.fit(X_train,y_train)
y_pred = regressor.predict(X_test)
mean_squared_error(y_test, y_pred)
mean_absolute_error(y_test, y_pred)

In [ ]:
# Pretty much finished.